## def put_call_parity(P_0, S_0, K, r, T):
    C_0 - P_0 = S_0 - K*np.exp(-r*T)
    C_0 : call à t_0
    P_0 : put à t_0
    S_0 : prix du sous-jacent à t_0
    K : strike price
    r : taux d'intérêt sans risque
    T : échéance 

## def black_scholes():
    C(S,t) = N(d_1)S - N(d_2)K*np.exp(-r*T)
    K : strike price
    S : Stock price à t
    r : taux d'échéance d'intérêt sans risque
    T : échéance
    t : temps

In [1]:
import matplotlib.pyplot as plt

%matplotlib inline
import numpy as np

from qiskit import QuantumCircuit
from qiskit_algorithms import IterativeAmplitudeEstimation, EstimationProblem
from qiskit.circuit.library import LinearAmplitudeFunction
from qiskit_aer.primitives import Sampler
from qiskit_finance.circuit.library import LogNormalDistribution

from scipy.stats import norm
import math
from math import exp, sqrt

In [2]:
r = [0.01,0.02,0.03,0.04,0.05,0.06,0.01,0.02,0.03,0.04,0.05,0.06,0.01,0.02,0.03,0.04,0.05,0.06,0.05,0.06]
K = [1,0.85,1.15,0.9,1.1,1.2,1,0.95,1.15,0.95,1.1,1.25,1,0.85,1.15,0.9,1.1,1.2,1.1,1.2]
S_0 = [1,1.1,0.9,0.95,1.05,1.25,1.1,1.1,0.87,0.95,1.07,1.25,1,1.1,0.99,0.957,1.05,1.45,1.05,1.25]
sigma = [0.05,0.15,0.2,0.1,0.125,0.09,0.057,0.17,0.19,0.13,0.175,0.14,0.4,0.31,0.2,0.25,0.15,0.09,0.156,0.099]
T = [1,2,3,4,5,6,1,2,3,4,5,6,1,2,3,4,5,6,5,6]
n_sim = np.full(20, 10000)
n_steps = np.full(20,5)


In [3]:
def black_Scholes_Put_Simulation(S0, K, r,  sigma, T, n_sim):
    mean = (r - 0.5 * sigma ** 2) * T
    std = sigma * np.sqrt(T)
    rng = np.random
    z = rng.normal(loc = mean, scale = std, size = n_sim)
    ST = S0 * np.exp(z)
    payoff = np.maximum(K - ST, 0.0)
    price = np.exp(-r * T) * payoff.mean()
    return float(price)

def black_Scholes_Call_Simulation(S0, K, r, sigma, T, n_sim):
    mean = (r - 0.5 * sigma ** 2) * T
    std = sigma * np.sqrt(T)
    rng = np.random
    z = rng.normal(loc = mean, scale = std, size = n_sim)
    ST = S0 * np.exp(z)
    payoff = np.maximum(ST - K, 0.0)
    price = np.exp(-r * T) * payoff.mean()
    return float(price)

In [4]:
for i in range(20):
    put_simulation = black_Scholes_Put_Simulation(S_0[i], K[i], r[i],  sigma[i], T[i], n_sim[i])
    call_simulation = black_Scholes_Call_Simulation(S_0[i], K[i], r[i],  sigma[i], T[i], n_sim[i])
    print(f"Put Simulation = {put_simulation}")
    print(f"Call Simulation = {call_simulation}")
    print("###############################################")

Put Simulation = 0.015418659438660347
Call Simulation = 0.025272279056133884
###############################################
Put Simulation = 0.007469884419112888
Call Simulation = 0.2947306695866524
###############################################
Put Simulation = 0.2264203017722774
Call Simulation = 0.07216545880507279
###############################################
Put Simulation = 0.012850083212281335
Call Simulation = 0.1954735551145786
###############################################
Put Simulation = 0.03635029407495529
Call Simulation = 0.22797652301216034
###############################################
Put Simulation = 0.0031760365995723242
Call Simulation = 0.4131463200338839
###############################################
Put Simulation = 0.0007453868638669285
Call Simulation = 0.11035259973076128
###############################################
Put Simulation = 0.02941798474031956
Call Simulation = 0.2136052552687145
###############################################
Put Simulatio

In [5]:
def black_scholes_call_binomial_tree(S0, K, r, sigma, T, n_steps):
    delta_t = T / n_steps
    u = np.exp(np.sqrt(delta_t) * sigma)
    d = np.exp(-np.sqrt(delta_t) * sigma)
    p = (np.exp(delta_t * r) - d) / (u - d)

    n_rows = 2 * n_steps + 1
    n_rows_mid = (n_rows + 1) // 2
    tree = np.zeros((n_rows, n_steps + 1))
    tree[n_rows_mid - 1, 0] = S0

    for i in range(1, n_steps + 1):
        old_idx_non_zero = np.where(tree[:, i - 1] != 0)[0]
        new_idx_non_zero = np.unique(np.concatenate((old_idx_non_zero + 1, old_idx_non_zero - 1)))
        new_idx_non_zero = new_idx_non_zero[(new_idx_non_zero >= 0) & (new_idx_non_zero < n_rows)]
        new_S_val = np.unique(S0 * (u ** (new_idx_non_zero - (n_rows_mid - 1))))
        tree[new_idx_non_zero, i] = new_S_val

    x = np.arange(0, n_steps + 1)
    vec_prob_payoff = np.array([math.comb(n_steps, k) * (1 - p) ** k * p ** (n_steps - k) for k in x])
    bool_payoff = np.array([True if i % 2 == 0 else False for i in range(2 * n_steps)])
    bool_payoff = np.append(bool_payoff, True)

    vec_payoff = np.maximum(tree[bool_payoff, n_steps] - K, 0)
    call_price_binomial_tree = np.exp(-T * r) * np.sum(vec_payoff * vec_prob_payoff)

    return {
        "mat_Binomial_Tree": tree,
        "vec_Payoff": vec_payoff,
        "vec_Prob_Payoff": vec_prob_payoff,
        "call_Price_Binomial_Tree": call_price_binomial_tree,
        "p": p,
        "d": d,
        "delta_t": delta_t
    }

def black_scholes_put_binomial_tree(S0, K, r, sigma, T, n_steps):
    delta_t = T / n_steps
    u = np.exp(np.sqrt(delta_t) * sigma)
    d = np.exp(-np.sqrt(delta_t) * sigma)
    p = (np.exp(delta_t * r) - d) / (u - d)

    n_rows = 2 * n_steps + 1
    n_rows_mid = (n_rows + 1) // 2
    tree = np.zeros((n_rows, n_steps + 1))
    tree[n_rows_mid - 1, 0] = S0

    for i in range(1, n_steps + 1):
        old_idx_non_zero = np.where(tree[:, i - 1] != 0)[0]
        new_idx_non_zero = np.unique(np.concatenate((old_idx_non_zero + 1, old_idx_non_zero - 1)))
        new_idx_non_zero = new_idx_non_zero[(new_idx_non_zero >= 0) & (new_idx_non_zero < n_rows)]
        new_S_val = np.unique(S0 * (u ** (new_idx_non_zero - (n_rows_mid - 1))))
        tree[new_idx_non_zero, i] = new_S_val

    x = np.arange(0, n_steps + 1)
    vec_prob_payoff = np.array([math.comb(n_steps, k) * (1 - p) ** k * p ** (n_steps - k) for k in x])
    bool_payoff = np.array([True if i % 2 == 0 else False for i in range(2 * n_steps)])
    bool_payoff = np.append(bool_payoff, True)

    vec_payoff = np.maximum(K - tree[bool_payoff, n_steps], 0)
    put_price_binomial_tree = np.exp(-T * r) * np.sum(vec_payoff * vec_prob_payoff)

    return {
        "mat_Binomial_Tree": tree,
        "vec_Payoff": vec_payoff,
        "vec_Prob_Payoff": vec_prob_payoff,
        "put_Price_Binomial_Tree": put_price_binomial_tree,
        "p": p,
        "d": d,
        "delta_t": delta_t
    }

In [22]:
put_binomials = []
call_binomials = []
for i in range(20):
    put_binomial = black_scholes_put_binomial_tree(S_0[i], K[i], r[i], sigma[i], T[i], n_steps[i])
    call_binomial = black_scholes_call_binomial_tree(S_0[i], K[i], r[i], sigma[i], T[i], n_steps[i])
    put_binomials.append(put_binomial)
    call_binomials.append(call_binomial)
    print(f"Put Binomial = {put_binomial["put_Price_Binomial_Tree"]}")
    print(f"Call Binomial = {call_binomial["call_Price_Binomial_Tree"]}")
    print("###############################################")

Put Binomial = 0.02456736585043369
Call Binomial = 0.017154489589642188
###############################################
Put Binomial = 0.009718086549383463
Call Binomial = 0.2550759672621381
###############################################
Put Binomial = 0.25842925480809725
Call Binomial = 0.054436455125652636
###############################################
Put Binomial = 0.08749031637604016
Call Binomial = 0.03562630736815229
###############################################
Put Binomial = 0.20090649341073896
Call Binomial = 0.026318844302805885
###############################################
Put Binomial = 0.21735350786412305
Call Binomial = 0.0035759360954889237
###############################################
Put Binomial = 0.0013459021305819556
Call Binomial = 0.09300785509187104
###############################################
Put Binomial = 0.03470275695188484
Call Binomial = 0.19765559019647982
###############################################
Put Binomial = 0.28131776049226775
Call B

In [24]:
for i in range(20):
    print("Put price:", put_binomials[i]["put_Price_Binomial_Tree"])
    print("Probability:", put_binomials[i]["vec_Prob_Payoff"])
    print("Payoffs:", put_binomials[i]["vec_Payoff"])
    print("Matrix bonimial tree:\n", put_binomials[i]["mat_Binomial_Tree"])

Put price: 0.02456736585043369
Probability: [0.04556574 0.19472387 0.33285873 0.28449242 0.12157701 0.02078223]
Payoffs: [0.10577996 0.06488152 0.02211253 0.         0.         0.        ]
Matrix bonimial tree:
 [[0.         0.         0.         0.         0.         0.89422004]
 [0.         0.         0.         0.         0.91444064 0.        ]
 [0.         0.         0.         0.93511848 0.         0.93511848]
 [0.         0.         0.9562639  0.         0.9562639  0.        ]
 [0.         0.97788747 0.         0.97788747 0.         0.97788747]
 [1.         0.         1.         0.         1.         0.        ]
 [0.         1.02261255 0.         1.02261255 0.         1.02261255]
 [0.         0.         1.04573643 0.         1.04573643 0.        ]
 [0.         0.         0.         1.06938321 0.         1.06938321]
 [0.         0.         0.         0.         1.09356469 0.        ]
 [0.         0.         0.         0.         0.         1.11829298]]
Put price: 0.009718086549383

In [25]:
for i in range(20):
    print("Call price:", call_binomials[i]["call_Price_Binomial_Tree"])
    print("Probability:", call_binomials[i]["vec_Prob_Payoff"])
    print("Payoffs:", call_binomials[i]["vec_Payoff"])
    print("Matrix bonimial tree:\n", call_binomials[i]["mat_Binomial_Tree"])

Call price: 0.017154489589642188
Probability: [0.04556574 0.19472387 0.33285873 0.28449242 0.12157701 0.02078223]
Payoffs: [0.         0.         0.         0.02261255 0.06938321 0.11829298]
Matrix bonimial tree:
 [[0.         0.         0.         0.         0.         0.89422004]
 [0.         0.         0.         0.         0.91444064 0.        ]
 [0.         0.         0.         0.93511848 0.         0.93511848]
 [0.         0.         0.9562639  0.         0.9562639  0.        ]
 [0.         0.97788747 0.         0.97788747 0.         0.97788747]
 [1.         0.         1.         0.         1.         0.        ]
 [0.         1.02261255 0.         1.02261255 0.         1.02261255]
 [0.         0.         1.04573643 0.         1.04573643 0.        ]
 [0.         0.         0.         1.06938321 0.         1.06938321]
 [0.         0.         0.         0.         1.09356469 0.        ]
 [0.         0.         0.         0.         0.         1.11829298]]
Call price: 0.255075967262

In [26]:
def black_Scholes_Call_Analytic(S_Zero, K, r, sigma, T):
    d1 = (np.log(S_Zero / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S_Zero * norm.cdf(d1) - K * np.exp(-r * (T)) * norm.cdf(d2)

def black_Scholes_Put_Analytic(S_Zero, K, r, sigma, T):
    d1 = (np.log(S_Zero / K) + (r + 0.5 * sigma ** 2) * (T)) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * (T)) * norm.cdf(-d2) - S_Zero * norm.cdf(-d1)

In [27]:
S_Zero = 1
for i in range(20):
    call_analytic = black_Scholes_Call_Analytic(S_Zero, K[i], r[i], sigma[i], T[i])
    put_analytic = black_Scholes_Put_Analytic(S_Zero, K[i], r[i], sigma[i], T[i])
    print(f"Put analytic = {put_analytic}")
    print(f"Call analytic = {call_analytic}")
    print("###############################################")

Put analytic = 0.015266236906369579
Call analytic = 0.025216403157201417
###############################################
Put analytic = 0.01733744284531044
Call analytic = 0.20066641956583575
###############################################
Put analytic = 0.16796578060239986
Call analytic = 0.11694491754048758
###############################################
Put analytic = 0.007504258234066932
Call analytic = 0.24057484816447672
###############################################
Put analytic = 0.046793984170220626
Call analytic = 0.19011312279167514
###############################################
Put analytic = 0.023907537576726506
Call analytic = 0.18669594629148922
###############################################
Put analytic = 0.017995605535292503
Call analytic = 0.02794577178612445
###############################################
Put analytic = 0.054364964095440005
Call analytic = 0.14161499690073298
###############################################
Put analytic = 0.16105757149043748
Call a